# Congressional Voting

**Kaggle: 184.702 TU ML WS 20**

**Goal: Predict the party of a congress member.**

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Get the Data

In [ ]:
data = pd.read_csv('./184702-tu-ml-ws-20-congressional-voting/CongressionalVotingID.shuf.lrn.csv')

In [ ]:
data

### Basic Data Information

In [ ]:
data.info()

In [ ]:
data.describe(include = 'object')

### Check missing values

In [ ]:
data.replace("unknown", np.nan, inplace = True)

In [ ]:
print(data.isnull().sum(axis=0))

In [ ]:
sns.heatmap(data.isnull(), yticklabels=False, cbar=False, cmap='viridis')

### Exploratory Data Analysis

**Class**

In [ ]:
sns.set_style('darkgrid')
ax = sns.countplot(x = data['class'])

total = len(data['class'])

for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:.1f}%'.format(100 * height/total),
            ha="center")